In [11]:
import pandas as pd
import bs4 as bs
import requests
df = pd.read_excel('Fin_Res_Q3_22_23_12_Feb_2023_in_V1.xls')
df = df.rename(columns=df.iloc[0]).drop(df.index[0])
df.head()

,symb,l_res_dt,sectr,res_url,REVENUE,PROFIT,DEPS,LUU,res_catg1,res_catg2
1,ARHAM,2023-01-04 00:00:00,Electronic - Miscellaneous,https://www.moneycontrol.com/financials/arhamt...,0,0,0,NaN,Stndl,Stndl
2,MAXIND,2023-01-04 00:00:00,Finance - Unique,https://www.moneycontrol.com/financials/maxind...,0,0,0,NaN,Stndl,Stndl
3,JBFIND,2023-01-09 00:00:00,Fibres & Plastics,http://www.moneycontrol.com/financials/jbfindu...,0,0,0,NaN,Stndl,Stndl
4,ANANDRATHI,2023-01-12 00:00:00,FINANCE - INVESTMENT,https://www.moneycontrol.com/financials/anandr...,0,0,0,NaN,Stndl,Stndl
5,CYIENT,2023-01-12 00:00:00,IT Consulting & Software,http://www.moneycontrol.com/financials/cyient/...,0,0,0,NaN,Stndl,Stndl


In [12]:
out_df = pd.DataFrame(columns=["SYMB", " DEC '22_R", " DEC '22_V", " DEC '22_DEPS",
                               " DEC '22_Interest", " DEC '22_Gross NPA", " DEC '22_Net NPA", "LUU",
                               "RES_CATG_1", "RES_CATG_2", "L_RES_DT", "DATA_AVAILABLE",
                               "Last Available Data", "TIME STAMP", "RES_URL", "SECTOR"])
out_df.head()
out_df["RES_CATG_1"] = df["res_catg1"]
out_df["RES_CATG_2"] = df["res_catg2"]
out_df["L_RES_DT"] = df["l_res_dt"]
out_df["SECTOR"] = df["sectr"]
out_df["RES_URL"] = df["res_url"]
out_df["SYMB"] = df["symb"]
out_df.head()

,SYMB,DEC '22_R,DEC '22_V,DEC '22_DEPS,DEC '22_Interest,DEC '22_Gross NPA,DEC '22_Net NPA,LUU,RES_CATG_1,RES_CATG_2,L_RES_DT,DATA_AVAILABLE,Last Available Data,TIME STAMP,RES_URL,SECTOR
1,ARHAM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stndl,Stndl,2023-01-04 00:00:00,NaN,NaN,NaN,https://www.moneycontrol.com/financials/arhamt...,Electronic - Miscellaneous
2,MAXIND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stndl,Stndl,2023-01-04 00:00:00,NaN,NaN,NaN,https://www.moneycontrol.com/financials/maxind...,Finance - Unique
3,JBFIND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stndl,Stndl,2023-01-09 00:00:00,NaN,NaN,NaN,http://www.moneycontrol.com/financials/jbfindu...,Fibres & Plastics
4,ANANDRATHI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stndl,Stndl,2023-01-12 00:00:00,NaN,NaN,NaN,https://www.moneycontrol.com/financials/anandr...,FINANCE - INVESTMENT
5,CYIENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stndl,Stndl,2023-01-12 00:00:00,NaN,NaN,NaN,http://www.moneycontrol.com/financials/cyient/...,IT Consulting & Software


In [13]:
urls = out_df["RES_URL"].tolist()
urls

['https://www.moneycontrol.com/financials/arhamtechnologies/results/quarterly-results/AT25#AT25',
 'https://www.moneycontrol.com/financials/maxindialimited/results/consolidated-quarterly-results/MIL19#MIL19',
 'http://www.moneycontrol.com/financials/jbfindustries/results/quarterly-results/JBF',
 'https://www.moneycontrol.com/financials/anandrathiwealth/results/consolidated-quarterly-results/ARW#ARW',
 'http://www.moneycontrol.com/financials/cyient/results/consolidated-quarterly-results/IE07',
 'http://www.moneycontrol.com/financials/dennetworks/results/consolidated-quarterly-results/DN02',
 'http://www.moneycontrol.com/financials/gmbreweries/results/quarterly-results/GMB#GMB',
 'http://www.moneycontrol.com/financials/infosys/results/consolidated-quarterly-results/IT',
 'https://www.moneycontrol.com/financials/abansholdings/results/quarterly-results/AH10#AH10',
 'https://www.moneycontrol.com/financials/alletechnologies/results/quarterly-results/AT26#AT26',
 'https://www.moneycontrol.com

In [36]:
def get_data(url):
    resp = requests.get(url)
    soup = bs.BeautifulSoup(resp.text, 'html.parser')
    table = soup.find('table', {'class': 'mctable1'})
    data = []
    if table is None:
        return data
    for row in table.findAll('tr'):
        cols = row.findAll('td')
        cols = [ele.text.strip() for ele in cols]
        print(cols)
        data.append([ele for ele in cols if ele])
    return data

In [28]:
# Main function to get data from all urls
def get_data_from_urls(urls):
    data = []
    for url in urls:
        data.extend(get_data(url))
    return data

In [32]:
get_data(urls[0])

[]

In [37]:
get_data(urls[1])

['Quarterly Results of Max India Limited (in Rs. Cr.)', "Dec '22", "Sep '22", "Jun '22", "Mar '22", "Dec '21", '']
['Net Sales/Income from operations', '43.06', '58.01', '43.61', '50.06', '41.69', '']
['Other Operating Income', '--', '--', '--', '--', '--', '']
['Total Income From Operations', '43.06', '58.01', '43.61', '50.06', '41.69', '']
['EXPENDITURE', '', '', '', '', '', '']
['Consumption of Raw Materials', '1.21', '0.87', '1.05', '0.92', '1.03', '']
['Purchase of Traded Goods', '--', '--', '--', '--', '--', '']
['Increase/Decrease in Stocks', '16.74', '22.47', '18.70', '23.92', '17.71', '']
['Power & Fuel', '--', '--', '--', '--', '--', '']
['Employees Cost', '15.30', '14.05', '14.26', '15.13', '12.15', '']
['Depreciation', '2.78', '2.49', '2.40', '2.33', '2.20', '']
['Excise Duty', '--', '--', '--', '--', '--', '']
['Admin. And Selling Expenses', '--', '--', '--', '--', '--', '']
['R & D Expenses', '--', '--', '--', '--', '--', '']
['Provisions And Contingencies', '--', '--', '

[['Quarterly Results of Max India Limited (in Rs. Cr.)',
  "Dec '22",
  "Sep '22",
  "Jun '22",
  "Mar '22",
  "Dec '21"],
 ['Net Sales/Income from operations',
  '43.06',
  '58.01',
  '43.61',
  '50.06',
  '41.69'],
 ['Other Operating Income', '--', '--', '--', '--', '--'],
 ['Total Income From Operations', '43.06', '58.01', '43.61', '50.06', '41.69'],
 ['EXPENDITURE'],
 ['Consumption of Raw Materials', '1.21', '0.87', '1.05', '0.92', '1.03'],
 ['Purchase of Traded Goods', '--', '--', '--', '--', '--'],
 ['Increase/Decrease in Stocks', '16.74', '22.47', '18.70', '23.92', '17.71'],
 ['Power & Fuel', '--', '--', '--', '--', '--'],
 ['Employees Cost', '15.30', '14.05', '14.26', '15.13', '12.15'],
 ['Depreciation', '2.78', '2.49', '2.40', '2.33', '2.20'],
 ['Excise Duty', '--', '--', '--', '--', '--'],
 ['Admin. And Selling Expenses', '--', '--', '--', '--', '--'],
 ['R & D Expenses', '--', '--', '--', '--', '--'],
 ['Provisions And Contingencies', '--', '--', '--', '--', '--'],
 ['Exp. C

In [27]:
out_df["DATA_AVAILABLE"] = get_data_from_urls(urls)
out_df.head()

out_df["Last Available Data"] = out_df["DATA_AVAILABLE"].apply(lambda x: x[-1])
out_df.head()
out_df["TIME STAMP"] = out_df["Last Available Data"].apply(lambda x: x[0])

out_df["DEC '22_R"] = out_df["Last Available Data"].apply(lambda x: x[1])

out_df["DEC '22_V"] = out_df["Last Available Data"].apply(lambda x: x[2])

out_df["DEC '22_DEPS"] = out_df["Last Available Data"].apply(lambda x: x[3])

out_df["DEC '22_Interest"] = out_df["Last Available Data"].apply(lambda x: x[4])

out_df["DEC '22_Gross NPA"] = out_df["Last Available Data"].apply(lambda x: x[5])

out_df["DEC '22_Net NPA"] = out_df["Last Available Data"].apply(lambda x: x[6])

out_df["LUU"] = out_df["Last Available Data"].apply(lambda x: x[7])

out_df.head()

out_df = out_df.drop(columns=["DATA_AVAILABLE", "Last Available Data"])
out_df.head()

out_df.to_csv("output.csv", index=False)

out_df.to_excel("output.xlsx", index=False)

out_df.to_json("output.json", orient="records")


AttributeError: 'NoneType' object has no attribute 'findAll'